<a href="https://colab.research.google.com/github/Luna-Yumiki/Interaction-between-AIs/blob/main2/Luna_Yumiki_5002FinalProject_CatDogClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import keras.utils as image
import numpy as np
from google.colab import files
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Demo_PetImages.zip" -d "/content/drive/MyDrive/"

Archive:  /content/drive/MyDrive/Demo_PetImages.zip
   creating: /content/drive/MyDrive/Demo_PetImages/
   creating: /content/drive/MyDrive/Demo_PetImages/Cat/
  inflating: /content/drive/MyDrive/Demo_PetImages/.DS_Store  
  inflating: /content/drive/MyDrive/__MACOSX/Demo_PetImages/._.DS_Store  
   creating: /content/drive/MyDrive/Demo_PetImages/Dog/
  inflating: /content/drive/MyDrive/Demo_PetImages/Cat/1409.jpg  
  inflating: /content/drive/MyDrive/__MACOSX/Demo_PetImages/Cat/._1409.jpg  
  inflating: /content/drive/MyDrive/Demo_PetImages/Cat/1353.jpg  
  inflating: /content/drive/MyDrive/__MACOSX/Demo_PetImages/Cat/._1353.jpg  
  inflating: /content/drive/MyDrive/Demo_PetImages/Cat/1435.jpg  
  inflating: /content/drive/MyDrive/__MACOSX/Demo_PetImages/Cat/._1435.jpg  
  inflating: /content/drive/MyDrive/Demo_PetImages/Cat/1421.jpg  
  inflating: /content/drive/MyDrive/__MACOSX/Demo_PetImages/Cat/._1421.jpg  
  inflating: /content/drive/MyDrive/Demo_PetImages/Cat/1347.jpg  
  inflati

Create folders to build the model.

In [ ]:
try:
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/cats/')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/dogs/')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/cats/')
    os.mkdir('/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/dogs/')
except OSError:
    pass

Split the data into Train and Validate folders.

In [ ]:
def split_data(SOURCE, TRAIN, VALIDATE, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " size is 0, so ignoring it.")

    train_length = int(len(files) * SPLIT_SIZE)
    validate_length = int(len(files) - train_length)
    shuffled_set = random.sample(files, len(files))
    train_set = shuffled_set[0:train_length]
    validate_set = shuffled_set[-validate_length:]

    for filename in train_set:
        this_file = SOURCE + filename
        destination = TRAIN + filename
        copyfile(this_file, destination)

    for filename in validate_set:
        this_file = SOURCE + filename
        destination = VALIDATE + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/content/drive/MyDrive/Demo_PetImages/Cat/"
TRAIN_CATS_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/cats/"
VALIDATE_CATS_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/cats/"
DOG_SOURCE_DIR = "/content/drive/MyDrive/Demo_PetImages/Dog/"
TRAIN_DOGS_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/dogs/"
VALIDATE_DOGS_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/dogs/"

split_size = .8
split_data(CAT_SOURCE_DIR, TRAIN_CATS_DIR, VALIDATE_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAIN_DOGS_DIR, VALIDATE_DOGS_DIR, split_size)

This is the model of Convolutional Neural Network architecture.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

Rescale and prepare data for the model building.

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/train/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=50,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/content/drive/MyDrive/Demo_PetImages/cats-v-dogs/validate/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=50,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


Train the model and evaluate the model iteratively.

In [ ]:
model_realization = model.fit(train_generator, epochs=5, steps_per_epoch=20,
                              validation_data=validation_generator, validation_steps=6)

Epoch 1/5
20/20 [==============================] - 6s 301ms/step - loss: 0.0805 - acc: 0.9730 - val_loss: 1.7620 - val_acc: 0.6033
Epoch 2/5
20/20 [==============================] - 7s 349ms/step - loss: 0.1702 - acc: 0.9380 - val_loss: 0.9410 - val_acc: 0.7133
Epoch 3/5
20/20 [==============================] - 7s 341ms/step - loss: 0.0568 - acc: 0.9810 - val_loss: 0.9725 - val_acc: 0.6267
Epoch 4/5
20/20 [==============================] - 7s 341ms/step - loss: 0.0531 - acc: 0.9870 - val_loss: 1.0409 - val_acc: 0.6933
Epoch 5/5
20/20 [==============================] - 8s 397ms/step - loss: 0.1729 - acc: 0.9470 - val_loss: 0.9302 - val_acc: 0.6900


Upload and test the images.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")

Saving Cat1.png to Cat1.png
Saving Cat2.png to Cat2.png
Saving Cat3.png to Cat3.png
Saving Cat4.png to Cat4.png
Saving CatButLikeDog1.png to CatButLikeDog1.png
Saving CatButLikeDog2.png to CatButLikeDog2.png
Saving CatButLikeDog3.png to CatButLikeDog3.png
Saving CatButLikeDog4.png to CatButLikeDog4.png
Saving Dog1.png to Dog1.png
Saving Dog2.png to Dog2.png
Saving Dog3.png to Dog3.png
Saving Dog4.png to Dog4.png
Saving DogButLikeCat1.png to DogButLikeCat1.png
Saving DogButLikeCat2.png to DogButLikeCat2.png
Saving DogButLikeCat3.png to DogButLikeCat3.png
Saving DogButLikeCat4.png to DogButLikeCat4.png
1/1 [==============================] - 0s 195ms/step
[1.]
Cat1.png is a dog
1/1 [==============================] - 0s 17ms/step
[1.]
Cat2.png is a dog
1/1 [==============================] - 0s 17ms/step
[1.]
Cat3.png is a dog
1/1 [==============================] - 0s 19ms/step
[1.]
Cat4.png is a dog
1/1 [==============================] - 0s 18ms/step
[1.]
CatButLikeDog1.png is a dog
1/1 [=